# Abstract
As the Black Lives Matter movement took root across the United States in the wake of George Floyd's death at the hands of Minneapolis police officers, a rallying cry has gained momentum in response to police brutality: defund the police. The movement doesn't aim to abolish lawn enforcement, but rather to redistribute the bulk of its funding to other areas.

Is such a notion even plausible? This notebook aims to draw a correlation between police brutality and its overfunding/militarization in areas with high rates of poverty, where other departments such as education and public health suffer from lack of funds.

## Imports

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# Autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

## Data

In [27]:
#Data import
pk = pd.read_csv('./data/police_killings.csv')
cd = pd.read_csv('./data/2018_census_5YE.csv')

#Data cleaning
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

pk.replace(to_replace=r'^Unknown race$', value='Unknown Race', regex=True, inplace=True)
cd.replace({'State': us_state_abbrev}, inplace=True)
cd = cd.set_index('State')
cd = cd.sort_index()

#Test
pk.head()
cd.head()

,Victim's name,Victim's age,Victim's gender,Victim's race,URL of image of victim,Date of Incident (month/day/year),Street Address of Incident,City,State,Zipcode,County,Agency responsible for death,Cause of death,A brief description of the circumstances surrounding the death,Official disposition of death (justified or other),Criminal Charges?,Link to news article or photo of official document,Symptoms of mental illness?,Unarmed,Alleged Weapon (Source: WaPo),Alleged Threat Level (Source: WaPo),Fleeing (Source: WaPo),Body Camera (Source: WaPo),WaPo ID (If included in WaPo database),Off-Duty Killing?,...,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65
0,Eric M. Tellez,28,Male,White,https://fatalencounters.org/wp-content/uploads...,31/12/2019,Broad St.,Globe,AZ,85501.0,Gila,Globe Police Department,Gunshot,"After midnight, a patrol officer was on routin...",Pending investigation,No known charges,https://www.azfamily.com/news/phoenix-man-arme...,No,Allegedly Armed,knife,other,not fleeing,no,5332.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name withheld by police,NaN,Male,Unknown Race,NaN,31/12/2019,7239-7411 I-40,Memphis,AR,38103.0,Crittenden,"Memphis Police Department, Arkansas State Police",Gunshot,"Police began a chase regarding a kidnapping, e...",Pending investigation,No known charges,https://www.fox16.com/local-news-2/kidnapping-...,No,Unclear,unclear,other,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Terry Hudson,57,Male,Black,NaN,31/12/2019,3600 N 24th St,Omaha,NE,68110.0,Douglas,Omaha Police Department,Gunshot,Police responded to a domestic incident on the...,Pending investigation,No known charges,https://www.ketv.com/article/omaha-police-offi...,No,Allegedly Armed,gun,attack,not fleeing,no,5359.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Malik Williams,23,Male,Black,NaN,31/12/2019,30800 14th Avenue South,Federal Way,WA,98003.0,King,Federal Way Police Department,Gunshot,Police responded to a domestic dispute. Police...,Pending investigation,No known charges,https://www.king5.com/article/news/local/2-fed...,No,Allegedly Armed,gun,attack,not fleeing,no,5358.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Frederick Perkins,37,Male,Black,NaN,31/12/2019,17057 N Outer 40 Rd,Chesterfield,MO,63005.0,St. Louis,Chesterfield Police Department,Gunshot,Police went to Chesterfield Outlets about 1 p....,Pending investigation,No known charges,https://www.stltoday.com/news/local/crime-and-...,No,Vehicle,vehicle,attack,car,no,5333.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Total Population,Male,Female,Hispanic,White,Black,Native American,Asian,Pacific Islander,Unknown Race
State,,,,,,,,,,
AK,"738,516","385,579","352,937","51,186","450,754","22,817","103,506","45,617","8,544","1,459"
AL,"4,864,680","2,355,799","2,508,881","203,146","3,196,730","1,285,737","23,243","63,936","1,521","7,503"
AR,"2,990,671","1,468,412","1,522,259","219,052","2,173,849","458,536","17,342","43,441","7,877","4,641"
AZ,"6,946,685","3,453,439","3,493,246","2,163,312","3,825,886","286,614","271,946","222,477","12,523","9,177"
CA,"39,148,760","19,453,769","19,694,991","15,221,577","14,695,836","2,164,519","138,427","5,525,439","138,911","97,763"


## Feature Engineering

In [33]:
#Functions
def race_data(data, race):
    new_data = data[data["Victim's race"] == race]
    sort_data = new_data[["Victim's name", "State"]]
    data_grouped = sort_data.groupby('State')["Victim's name"].nunique()
    data_df = data_grouped.to_frame()
    data_df = data_df.rename(columns = {"Victim's name" : race + ' Police Killings'})
    return data_df

#Police killings total by race per state
hispanic_df = race_data(pk, 'Hispanic')
black_df = race_data(pk, 'Black')
white_df = race_data(pk, 'White')
asian_df = race_data(pk, 'Asian')
native_df = race_data(pk, 'Native American')
pacific_df = race_data(pk, 'Pacific Islander')
unknown_df = race_data(pk, 'Unknown Race')

#Police killings total across all races per state
pk_total = pk[["Victim's name", 'State']]
state_total = pk_total.groupby('State')["Victim's name"].nunique()
state_total_df = state_total.to_frame()
state_total_df = state_total_df.rename(columns = {"Victim's name": 'Total Police Killings'})

#Combining data into single df
hispanic_df['Total Police Killings'] = state_total_df['Total Police Killings']
hispanic_df[['Hispanic Population', 'Total State Population']] = cd[['Hispanic', 'Total Population']]
black_df['Total Police Killings'] = state_total_df['Total Police Killings']
black_df[['Black Population', 'Total State Population']] = cd[['Black', 'Total Population']]
white_df['Total Police Killings'] = state_total_df['Total Police Killings']
white_df[['White Population', 'Total State Population']] = cd[['White', 'Total Population']]
asian_df['Total Police Killings'] = state_total_df['Total Police Killings']
asian_df[['Asian Population', 'Total State Population']] = cd[['Asian', 'Total Population']]
native_df['Total Police Killings'] = state_total_df['Total Police Killings']
native_df[['Native American Population', 'Total State Population']] = cd[['Native American', 'Total Population']]
pacific_df['Total Police Killings'] = state_total_df['Total Police Killings']
pacific_df[['Pacific Islander Population', 'Total State Population']] = cd[['Pacific Islander', 'Total Population']]
unknown_df['Total Police Killings'] = state_total_df['Total Police Killings']
unknown_df[['Unknown Race Population', 'Total State Population']] = cd[['Unknown Race', 'Total Population']]

#Test
hispanic_df.head()
black_df.head()
white_df.head()
asian_df.head()
native_df.head()
pacific_df.head()
unknown_df.head()
print(pk_total)
print(state_total)
state_total_df.head()

,Hispanic Police Killings,Total Police Killings,Hispanic Population,Total State Population
State,,,,
AK,1,41,"51,186","738,516"
AR,3,103,"219,052","2,990,671"
AZ,114,338,"2,163,312","6,946,685"
CA,469,1123,"15,221,577","39,148,760"
CO,67,218,"1,184,794","5,531,141"


,Black Police Killings,Total Police Killings,Black Population,Total State Population
State,,,,
AK,5,41,"22,817","738,516"
AL,51,136,"1,285,737","4,864,680"
AR,28,103,"458,536","2,990,671"
AZ,31,338,"286,614","6,946,685"
CA,184,1123,"2,164,519","39,148,760"


,White Police Killings,Total Police Killings,White Population,Total State Population
State,,,,
AK,15,41,"450,754","738,516"
AL,73,136,"3,196,730","4,864,680"
AR,59,103,"2,173,849","2,990,671"
AZ,141,338,"3,825,886","6,946,685"
CA,331,1123,"14,695,836","39,148,760"


,Asian Police Killings,Total Police Killings,Asian Population,Total State Population
State,,,,
AL,1,136,"63,936","4,864,680"
AR,1,103,"43,441","2,990,671"
CA,44,1123,"5,525,439","39,148,760"
CO,4,218,"169,556","5,531,141"
CT,1,36,"157,406","3,581,504"


,Native American Police Killings,Total Police Killings,Native American Population,Total State Population
State,,,,
AK,12,41,"103,506","738,516"
AZ,14,338,"271,946","6,946,685"
CA,7,1123,"138,427","39,148,760"
CO,5,218,"30,131","5,531,141"
ID,2,48,"18,775","1,687,809"


,Pacific Islander Police Killings,Total Police Killings,Pacific Islander Population,Total State Population
State,,,,
CA,9,1123,"138,911","39,148,760"
HI,22,37,"132,583","1,422,029"
MI,1,114,"2,464","9,957,488"
MO,2,194,"6,037","6,090,062"
NC,1,204,"5,677","10,155,624"


,Unknown Race Police Killings,Total Police Killings,Unknown Race Population,Total State Population
State,,,,
AK,8,41,"1,459","738,516"
AL,12,136,"7,503","4,864,680"
AR,12,103,"4,641","2,990,671"
AZ,39,338,"9,177","6,946,685"
CA,83,1123,"97,763","39,148,760"


                Victim's name State
0              Eric M. Tellez    AZ
1     Name withheld by police    AR
2                Terry Hudson    NE
3              Malik Williams    WA
4           Frederick Perkins    MO
...                       ...   ...
7902                      NaN   NaN
7903                      NaN   NaN
7904                      NaN   NaN
7905                      NaN   NaN
7906                      NaN   NaN

[7907 rows x 2 columns]
State
AK      41
AL     136
AR     103
AZ     338
CA    1123
      ... 
VT      12
WA     210
WI     111
WV      72
WY      19
Name: Victim's name, Length: 51, dtype: int64


,Total Police Killings
State,
AK,41
AL,136
AR,103
AZ,338
CA,1123


# Analysis/Modeling
Do work here

In [5]:
#Functions
state_sum = pk.groupby(['State', "Victim's race"]).count()
def create_pie_chart(input_data, state):
    labels = input_data.loc[state]["Victim's name"].index
    values = input_data.loc[state]["Victim's name"]
    trace = go.Pie(labels = labels, values = values)
    data = [trace]
    fig = go.Figure(data = data)
    iplot(fig)

create_pie_chart(state_sum, 'NJ')
create_pie_chart(state_sum, 'NY')
create_pie_chart(state_sum, 'CA')

# Results
Show graphs and stats here

# Conclusions and Next Steps
Summarize findings here